## Analysis of Robinhood Portfolio (BeautifulSoup)

Project by: Pete Aguirre II

In this project, I will do a retuern:risk analysis on my current Robinhood stock portfolio with the help of multiple 
tools using:
- Python 3
- Jupyter Lab/Notebook
- Beautiful Soup
- Markowitz Efficent Frontier

In [1]:
# Libraries Used 
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import sys

import robin_stocks as r 
import pyotp
import json

from pandas_datareader import data as wb
from bs4 import BeautifulSoup as soup


# Magic
%matplotlib inline

## For manually obtained CSV

In [ ]:
my_portfolio = pd.read_csv('hypothetical_portfolio.csv')
my_portfolio

In [ ]:
#dollar_sign = ['Average Cost', 'Total Return', 'Equity']
#for d in dollar_sign:
#    my_portfolio[d] = pd.to_numeric(my_portfolio[d])
my_portfolio.info()

In [ ]:
# Only run to download weights
total_sum = (my_portfolio['Average Cost']*my_portfolio['Shares']).sum()
weights = []
for i in range(len(my_portfolio)):
    weights.append(round((my_portfolio['Average Cost'][i]*my_portfolio['Shares'][i])/total_sum, 2))
    
#my_portfolio = pd.DataFrame({'Name':names_txt, 'Symbol':tick_new, 'Shares':shares_txt, 'Average Cost':avg_txt, 'Total Return':returns_txt, 'Equity':equity_txt, 'Weight':weights})
weights = pd.DataFrame({'Weights': weights})
weights.to_csv('weights.csv', index=False)


## 3.) Data Collection (Historical Prices)

In [ ]:
# Collect historical prices
tickers = my_portfolio['Symbol']
start_date = '2000-01-01'
my_data = pd.DataFrame()
for t in tickers:
    my_data[t] = wb.DataReader(t, data_source='yahoo', start=start_date)['Adj Close']

In [ ]:
my_data = my_data.dropna()
my_data.info()

In [ ]:
sec_returns = np.log(my_data/my_data.shift(1))
sec_returns

sec_returns.info()

In [ ]:
sec_returns.head()

In [ ]:
sec_returns.tail()

## Data Analysis

In [ ]:
sec_returns

In [ ]:
for t in tickers:

    
    print("")
    print("#",t)
    print("DAILY")
    print("Daily Return:", round(sec_returns[t].mean()*100, 4), "%")
    print("Daily Risk:", round(sec_returns[t].std()*100, 4), "%")

    print("")

    print("ANNUAL")
    mean_return = round((sec_returns[t].mean()*250)*100, 4)
    std_return = round((sec_returns[t].std()*250**.5)*100, 4)
    print("Annual Return:", mean_return, "%")
    print("Annual Risk:", std_return, "%")
    

    
    

In [ ]:
sec_returns.idxmin() 

In [ ]:
sec_returns.idxmax()

In [ ]:
# Covariance 
# Determines if there is a relationship between two stocks, whether they move together:
# positively, negatively, or neutral 
return_cov = sec_returns.cov()
return_cov

In [ ]:
# Covariance 
# Determines if there is a relationship between two stocks, whether they move together:
# positively, negatively, or neutral 
return_cov = sec_returns.cov()*250
return_cov

In [ ]:
# Correlations
# Determines how close the relationship are between two stocks
return_corr = sec_returns.corr()
return_corr

In [ ]:
# Calculating Diversifiable and Non-Diversifiable

# Portfolio variance
pflio_var = np.dot(my_portfolio['Weight'].T, np.dot(sec_returns.cov()*250, my_portfolio['Weight']))
print("Portfolio Variance", pflio_var)

# Portfolio volatility 
#pflio_vol = (np.dot(my_portfolio['Weight'].T, np.dot(sec_returns.cov()*250, my_portfolio['Weight'])))**.5
#print("Portfolio Volatility:", pflio_vol)

# or...

pflio_vol2 = np.sqrt(pflio_var)
print("Portfolio Volatility:", pflio_vol2)


# Calculating variance annually for each tickers
var_a = []
for t in tickers:
    var_a.append(sec_returns[t].var()*250)

var_a = np.array(var_a)
variances = pd.DataFrame({'Name':names_txt, 'Variance':var_a})
print("")
# Diversifiable Risk
# div_risk = pflio_var - my_portfolio['Weight'][0]**2*variances['Variance'][0] - ...
#                  ... - my_portfolio['Weight'][n]**2*variances['Variance'][n]
div_risk = 0   
for i in range(len(my_portfolio)):
    if i==0:
        div_risk = pflio_var - my_portfolio['Weight'][0]**2*variances['Variance'][0]
    else:
        div_risk -= my_portfolio['Weight'][i]**2*variances['Variance'][i]
print("Diversifiable Risk:", div_risk)

print("")
# Non Diversifiable Risk
non_div_risk1 = pflio_var - div_risk
print("Non-Diversifiable Risk:", non_div_risk1)

print("")
non_div_risk2 = 0
for i in range(len(my_portfolio)):
    non_div_risk2 += my_portfolio['Weight'][i]**2*variances['Variance'][i]
    #print(my_portfolio['Name'][i], non_div_risk2)

print("")
print("Non-Diversifiable Risk:", non_div_risk2)
print("Non-Diversifiable Risk:", non_div_risk2 == non_div_risk1)


##test = my_portfolio['Weight'].diff()
#print(test)

In [ ]:
# FOR THE MANUAL MODE
# Calculating Diversifiable and Non-Diversifiable

# Portfolio variance
pflio_var = np.dot(my_portfolio['Weight'].T, np.dot(sec_returns.cov()*250, my_portfolio['Weight']))
print("Portfolio Variance", pflio_var)

# Portfolio volatility 
pflio_vol = (np.dot(my_portfolio['Weight'].T, np.dot(sec_returns.cov()*250, my_portfolio['Weight'])))**.5
print("Portfolio Volatility:", pflio_vol)

# Calculating variance annually for each tickers
var_a = []
for t in tickers:
    var_a.append(sec_returns[t].var()*250)

var_a = np.array(var_a)
variances = pd.DataFrame({'Name':my_portfolio['Name'], 'Variance':var_a})

# Diversifiable Risk
# div_risk = pflio_var - my_portfolio['Weight'][0]**2*variances['Variance'][0] - ...
#                  ... - my_portfolio['Weight'][n]**2*variances['Variance'][n]
div_risk = 0   
div_risk = pflio_var - my_portfolio['Weight'][0]**2*variances['Variance'][0]
for i in range(len(my_portfolio)-1):
    div_risk -= my_portfolio['Weight'][i+1]**2*variances['Variance'][i+1]
    #print(my_portfolio['Name'][i], div_risk)
div_risk = div_risk - my_portfolio['Weight'][len(my_portfolio)-1]**2*variances['Variance'][len(my_portfolio)-1]
print("Diversifiable Risk:", div_risk)

# Non Diversifiable Risk
non_div_risk1 = pflio_var - div_risk
print("Non-Diversifiable Risk:", non_div_risk1)

non_div_risk2 = 0
for i in range(len(my_portfolio)):
    non_div_risk2 += my_portfolio['Weight'][i]**2*variances['Variance'][i]
print("Non-Diversifiable Risk:", non_div_risk2)
print("Non-Diversifiable Risk:", non_div_risk2 == non_div_risk1)


##test = my_portfolio['Weight'].diff()
#print(test)

In [ ]:
no_assets = len(tickers)
no_assets

In [ ]:
pflio_ret = []
pflio_vol = [] 
pflio_wei = []

for x in range(10000):
    weights = np.random.random(no_assets)
    weights /= np.sum(weights)
    
    pflio_wei.append(weights)
    pflio_ret.append(np.sum(weights*sec_returns.mean())*250)
    pflio_vol.append(np.sqrt(np.dot(weights.T, np.dot(sec_returns.cov()*250, weights))))
    #print(x, weights)

pflio_wei = np.array(pflio_wei)
pflio_ret = np.array(pflio_ret)
pflio_vol = np.array(pflio_vol)

#print(np.sum(weights))
#pflio_returns, pflio_volatilities

In [ ]:
pflio_scenarios = pd.DataFrame({'Return': pflio_ret, 'Volatility': pflio_vol})
pflio_scenarios = pflio_scenarios.sort_values('Return', ascending=True)

In [ ]:
pflio_scenarios.head()
#pflio_scenarios['Return']

In [ ]:
pflio_scenarios.tail(3500)

In [ ]:
# Efficient Frontier 
pflio_scenarios.plot(x='Volatility', y='Return', kind='scatter', figsize=(10,6));
plt.xlabel('Expected Volatility')
plt.ylabel('Expected Return')
eff_front = plt.savefig("efficient_frontier2.png")

In [ ]:
# For Testing purposes
pfolio_wei2 = []
tickers2 = []
#print(pflio_wei[3273])
tickers2 = np.array(tickers)
#print(tickers2)
#print(np.sum(pflio_wei[3273]))
pflio_wei2 = pflio_wei[3273]


#ideal_portfolio = pd.DataFrame(columns=tickers2)
#ideal_portfolio = pd.DataFrame({'Symbol': tickers2, 'Weights': pflio_wei2})
ideal_portfolio = pd.DataFrame({'Weights': pflio_wei2})
ideal_portfolio = pd.concat([symbols, ideal_portfolio], axis=1)
ideal_portfolio.to_csv('ideal_weights.csv', index=False)
ideal_portfolio

## Data Visualization

In [ ]:
(my_data/my_data.iloc[0]*100).plot(figsize=(20,15))

In [ ]:
sns.set_style('whitegrid')
regression = sns.pairplot(sec_returns[1:], kind="reg")
regression = regression.savefig("regression.png")
regression

In [ ]:
plt.subplots(figsize=(15, 15))
heatmap = sns.heatmap(return_corr, annot=True, square=True, cmap='coolwarm')
heatmap.savefig("heatmap.png")
heatmap
#plt.show()